In [112]:
import math
import numpy as np
import pandas as pd
import os

#Calculate the distance between two location give lat and lng
EARTH_RADIUS = 6378.137
def rad(d):
    return d * math.pi / 180.0;

def GetDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1);
    radLat2 = rad(lat2);
    a = radLat1 - radLat2;
    b = rad(lng1) - rad(lng2);

    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2),2) +
    math.cos(radLat1)*math.cos(radLat2)*math.pow(math.sin(b/2),2)));
    s = s * EARTH_RADIUS;
    return s * 0.621371;
yelp = pd.read_csv("newdata.csv",)
all_points = yelp[['latitude', 'longitude']].values


# The number of competitors for each restaraunt.
count_neighbor = []
for index, row in yelp.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    count = 0
    for others in all_points:
        dis = GetDistance(lat, lng, others[0], others[1])
        # Count plus one if distance less than 0.5 mile
        if dis < 0.5:
            count += 1
    count_neighbor.append(count - 1)


In [113]:
# add number of subway station around(in 0.2 mile)

# get all subway station location
subway = pd.read_csv('../rawdata/subway_entrance/subway entrance.csv')
stations = []
for index, row in subway.iterrows():
    raw = row["the_geom"].split(" ")
    lat = float(raw[2][:-1])
    lng = float(raw[1][1:])
    stations.append([lat, lng])


In [114]:
# Calculate distance to all subway stations for each restaraunt.
count_subway_around = []
for index, row in yelp.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    count = 0
    for others in stations:
        dis = GetDistance(lat, lng, others[0], others[1])
        if dis < 0.2:
            count += 1
    count_subway_around.append(count)
    
yelp['subway_around'] = np.array(count_subway_around)

In [115]:
# target variable 
# 1. profit = revenue - cost = lambda * price * checkin - price
# 2. checkin rate = lambda * review count
# 3. rate in yelp = score in yelp

# cost_dic = pd.read_csv("./address_price/mean_on_zip")

# Get the map key: zipcode, value: cost
# Get mean cost for all area (some zipcode area don't have cost data)
price = pd.read_csv("../address_price/mean_on_zip.csv")
cost_dic = dict(zip(price.zipcode, price.average_price))
mean_cost = 2.5


# assume the constant for target variable 1 now
lambda1 = 1
target1 = []
for index, row in yelp.iterrows():
    if row['zip code'] not in cost_dic:
        cost = mean_cost
    else:
        cost = cost_dic[row['zip code']]

    if str(row['price']).strip() == 'nan':
        pri = mean_cost
    else:
        pri = len(row['price'])
    target1.append(lambda1 * pri * row['review count'] - cost)


In [116]:
# Checkin number = lambda2 * review count
lambda2 = 10
target2 = []
for index, row in yelp.iterrows():
    target2.append(lambda2 * row['review count'])

In [117]:
# Rate in yelp
target3 = []
for index, row in yelp.iterrows():
    target3.append(row['rating'])

In [118]:
def norm(x):
    x = np.array(x)
    return (x - x.min(0)) / x.ptp(0)

yelp['target1'] = np.array(target1)
yelp['target1_norm'] = norm(target1)
yelp['target2'] = np.array(target2)
yelp['target2_norm'] = norm(target2)
yelp['target3'] = np.array(target3)
yelp['target3_norm'] = norm(target3)

In [120]:
yelp.to_csv('target_variable.csv')
yelp

,name,price,rating,zip code,latitude,longitude,review count,subway_around,target1,target1_norm,target2,target2_norm,target3,target3_norm
0,Dim Sum Palace,$$,4.0,10036.0,40.760150,-73.989370,493,10,918.674444,0.124446,4930,0.107073,4.0,0.750
1,Wah Fung No 1,$,4.5,10002.0,40.717350,-73.994570,819,4,684.500000,0.100562,8190,0.178020,4.5,0.875
2,Tasty Garden,$,5.0,10009.0,40.724850,-73.983060,16,0,-102.772500,0.020265,160,0.003264,5.0,1.000
3,Peng's Noodle Folk,$$,4.5,10028.0,40.777260,-73.949320,174,4,98.000000,0.040743,1740,0.037650,4.5,0.875
4,Zest Szechuan,$$,4.0,10018.0,40.752740,-73.984450,240,16,407.705652,0.072331,2400,0.052013,4.0,0.750
5,Tuome,$$$,4.5,10009.0,40.724125,-73.982891,290,0,751.227500,0.107367,2900,0.062894,4.5,0.875
6,Shu Jiao Fu Zhou Cuisine Restaurant,$,4.5,10002.0,40.718429,-73.991693,387,9,252.500000,0.056501,3870,0.084004,4.5,0.875
7,China Jade,$$,4.5,10028.0,40.777470,-73.952390,19,4,-212.000000,0.009125,190,0.003917,4.5,0.875
8,China Xiang,$$,4.0,10036.0,40.758290,-73.992511,235,8,402.674444,0.071817,2350,0.050925,4.0,0.750
9,Han Dynasty,$$,4.0,10024.0,40.787520,-73.976470,339,5,541.520000,0.085979,3390,0.073558,4.0,0.750
